In [1]:
import torch
import gpytorch
from data import PVWeatherLoader


In [2]:
# data parameters
DAY_INIT = 40
DAY_MIN = 8
DAY_MAX = 16
N_DAYS = 10
MINUTE_INTERVAL = 60
DAILY_DATA_POINTS = (DAY_MAX - DAY_MIN) * 60 / MINUTE_INTERVAL
N_SYSTEMS = 10
# create a tuple of 4 coordinates that form a polygon in the uk
# and a circle with a radius of r
CIRCLE_COORDS = (55, -1.5)
RADIUS = 0.3
POLY_COORDS = ((50, -6), (50.5, 1.9), (57.6, -5.5), (58, 1.9))

In [3]:
loader = PVWeatherLoader(
    # number of days to get data for
    n_days=N_DAYS,
    # initial day of the data for that season
    # look at the data frame to see which day it is
    day_init=DAY_INIT,
    # number of systems to extract
    n_systems=N_SYSTEMS,
    coords=CIRCLE_COORDS,
    radius=RADIUS,
    # the minute interval our data is sampled at 
    # (e.g. 60 for hourly, 30 for half hourly, 15 for 15 minutes, etc.)
    minute_interval=MINUTE_INTERVAL,
    # the minimum and maximum hour of the day to use
    # (e.g. 8 and 15 for 8am to 3pm)
    day_min=DAY_MIN,
    day_max=DAY_MAX,
    folder_name='pv_data',
    file_name='pv_and_weather.csv',
    distance_method='circle',
    # optionally use a season
    season='winter',
    # optionally drop series with nan values
    drop_nan=False
)

==> Loading data
==> Loaded data in: 0 m : 0 sec



/Users/eirikbaekkelund/Desktop/UCL/Code/Dissertation/data/utils.py:547: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['season'], axis=1, inplace=True)


In [4]:
from data.utils import train_test_split

for X, y in loader:
    x_train, x_test, y_train, y_test = train_test_split(X, y, hour=8, minute_interval=60, day_min=8, day_max=15, n_hours=8)
    print(X.shape, y.shape)

torch.Size([80, 5]) torch.Size([80])
torch.Size([80, 5]) torch.Size([80])
torch.Size([80, 5]) torch.Size([80])
torch.Size([80, 5]) torch.Size([80])
torch.Size([80, 5]) torch.Size([80])
torch.Size([80, 5]) torch.Size([80])
torch.Size([80, 5]) torch.Size([80])
torch.Size([80, 5]) torch.Size([80])


In [5]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

(torch.Size([66, 5]), torch.Size([8, 5]), torch.Size([66]), torch.Size([8]))

In [8]:
import torch
import gpytorch
import numpy as np
import wandb
from gpytorch.variational import (VariationalStrategy, 
                                  LMCVariationalStrategy,
                                  IndependentMultitaskVariationalStrategy,
                                  MeanFieldVariationalDistribution)
from gpytorch.models import ApproximateGP
from gpytorch.distributions import MultivariateNormal
from data.utils import store_gp_module_parameters
from typing import Optional

class MultitaskGPModel(ApproximateGP):
    def __init__(self,
                 X : torch.Tensor,
                 y : torch.Tensor,
                 likelihood : gpytorch.likelihoods.Likelihood = None,
                 mean_module : gpytorch.means.Mean = None,
                 covar_module : gpytorch.kernels.Kernel = None,
                 num_latents : int = 1,
                 learn_inducing_locations : bool = False,
                 jitter : float = 1e-4,
                 task_indices : Optional[torch.Tensor] = None):
        # check that num_latents is consistent with the batch_shape of the mean and covar modules
        assert num_latents == mean_module.batch_shape[0], 'num_latents must be equal to the batch_shape of the mean module'
        assert num_latents == covar_module.batch_shape[0], 'num_latents must be equal to the batch_shape of the covar module'
        if task_indices is not None:
            assert num_latents == task_indices.max() + 1, 'num_latents must be equal to the number of tasks'
            self.task_indices = task_indices

        num_tasks = y.size(-1)
        
        # MeanField constructs a variational distribution for each output dimension
        variational_distribution = MeanFieldVariationalDistribution(
                                    num_inducing_points=X.size(0), 
                                    batch_shape=torch.Size([num_latents]),
                                    jitter=jitter
                                )
        
        # LMC constructs MultitaskMultivariateNormal from the base var dist
        variational_strategy = IndependentMultitaskVariationalStrategy(
                            VariationalStrategy(
                                    model=self, 
                                    inducing_points=X, 
                                    variational_distribution=variational_distribution, 
                                    learn_inducing_locations=learn_inducing_locations,
                                ),
                            num_tasks=num_tasks,
                            task_dim=-1
                        )
        
        super().__init__(variational_strategy)
        
        self.mean_module = mean_module
        self.covar_module =  covar_module
        self.likelihood = likelihood
        self.X = X
        self.y = y
        
    def forward(self, x,):
    
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
      
        return MultivariateNormal(mean_x, covar_x)
    
    def fit(self, 
            n_iter : int,
            lr : float, 
            verbose : bool = False,
            use_wandb : bool = False):
            
        self.train()
        self.likelihood.train()

        losses = []

        if use_wandb:
            wandb.init(
                project ='dissertation',
                config={'learning_rate': lr, 'epochs': n_iter}
            )
        
        mll = gpytorch.mlls.VariationalELBO(self.likelihood, self, num_data=self.y.size(0))
        optim = torch.optim.Adam(self.parameters(), lr=lr)
        
        print_freq = n_iter // 10
        
        for i in range(n_iter):
            
            optim.zero_grad()
            output = self(self.X, task_indices=self.task_indices)
            loss = -mll(output, self.y, task_indices=self.task_indices)
            loss.backward()
            optim.step()

            losses.append(loss.item())
            
            if verbose and (i+1) % print_freq == 0:
                print(f'Iter {i+1}/{n_iter} - Loss: {loss.item()}')
            
            if use_wandb:
                log_dict = store_gp_module_parameters(self)
                log_dict['loss'] = loss.item()
                wandb.log(log_dict)
            
            # if loss is not decreasing for 15 iterations, stop training
            if i > 0:
                if abs(losses[-2] - losses[-1]) < 1e-6:
                    j += 1
                    if j == 15:
                        print(f'Early stopping at iter {i+1}')
                        break
                else:
                    j = 0
        
        if use_wandb:
            wandb.finish()
    
    def get_inducing_points(self):
        return self.variational_strategy.base_variational_strategy.inducing_points
    
    def predict_mean(self, dist):
        return dist.mean.mean(axis=0)
    
    def predict_mode(self):
        return self.likelihood.mode().mean(axis=0)
    
    def predict_median(self, samples):
        return samples.median(axis=0).values.mean(axis=0)
    
    def confidence_region(self, samples):
        # per MC sample
        lower, upper = np.percentile(samples, [2.5, 97.5], axis=0)
        # across tasks
        lower, upper = lower.mean(axis=0), upper.mean(axis=0)
        return lower, upper

    def predict(self, x, pred_type='dist'):
        """ 
        Get the predictions for the given x values.
        The prediction type can be one of: dist, median, mean, mode, all or
        one can get the posterior predictive distribution.

        Args:
            x (torch.Tensor): input tensor
            pred_type (str, optional): prediction type. Defaults to 'median'.
        
        Returns:
            dist (torch.distributions.Distribution) if pred_type is 'dist': the posterior predictive distribution
            (pred, lower, upper) (torch.Tensor, torch.Tensor, torch.Tensor) if pred_type is 'median', 'mean', 'mode'
            
            where pred is the prediction of the given type and lower, upper 
            is the 95% confidence interval from MC sampling from the predictive distribution.
    
        """
        assert pred_type in ['dist', 'median', 'mean', 'mode', 'all'], 'pred_type must be one of: dist, median, mean, mode, all'
    
        if isinstance(self.likelihood, gpytorch.likelihoods.MultitaskGaussianLikelihood):
            with torch.no_grad(), gpytorch.settings.fast_pred_var():
                dist = self.likelihood(self(x))
                mean = dist.mean
                lower, upper = dist.confidence_region()

                return mean, lower, upper
        
        elif isinstance(self.likelihood, gpytorch.likelihoods.BetaLikelihood):
           with torch.no_grad(), gpytorch.settings.fast_pred_var(), gpytorch.settings.num_likelihood_samples(30):
                
                dist = self.likelihood(self(x))
                if pred_type == 'dist':
                    return dist

                samples = dist.sample(sample_shape=torch.Size([30]))
                lower, upper = self.confidence_region(samples)

                if pred_type == 'median':
                    median = self.predict_median(samples)
                    return median, lower, upper
                
                elif pred_type == 'mean':
                    mean = self.predict_mean()
                    return mean, lower, upper
                
                elif pred_type == 'mode':
                    mode = self.predict_mode()
                    return mode, lower, upper
             
                else:
                    median = self.predict_median(samples)
                    mean = self.predict_mean()
                    mode = self.predict_mode()
                    
                    return median, mean, mode, lower, upper
                    
        else:
            raise NotImplementedError('Likelihood not implemented')



In [24]:
from gpytorch.constraints import Positive
from kernels import Kernel
from likelihoods import BetaLikelihood_MeanParametrization, MultitaskBetaLikelihood
from gpytorch.means import ZeroMean


# input for hadamard model
dict_input = {'input' : [],
              'output' : [],
              'task_indices' : []}

# containts the data for each task at same time intervals
for i, (X, y) in enumerate(loader):
    n = X.shape[0]
    dict_input['input'].append(X)
    dict_input['task_indices'].append(torch.ones(n, dtype=torch.long) * i)
    dict_input['output'].append(y)

task_indices = torch.cat(dict_input['task_indices'])
x = torch.cat(dict_input['input'], dim=0)
y = torch.stack(dict_input['output'], dim=-1)
num_latents = y.size(-1)


kernel = Kernel(num_latent=num_latents)
matern_base = kernel.get_matern(lengthscale_constraint=Positive(),
                                outputscale_constraint=Positive())
matern_quasi = kernel.get_matern(lengthscale_constraint=Positive(),
                                 outputscale_constraint=Positive())
periodic1 = kernel.get_periodic(lengthscale_constraint= Positive(),
                                outputscale_constraint=Positive())
periodic2 = kernel.get_periodic(lengthscale_constraint= Positive(),
                                outputscale_constraint=Positive()
                                )

covar_module = kernel.get_quasi_periodic(matern_base=matern_base, 
                                    matern_quasi=matern_quasi,
                                    periodic1=periodic1,
                                    periodic2=None)

likelihood = MultitaskBetaLikelihood(num_tasks=num_latents)
mean_module = ZeroMean(batch_shape=torch.Size([num_latents]))

# TODO should have task indices specified in forward method
# TODO should have index kernel for task covariance?
# TODO one dim or multi dim likelihood?
# TODO independent multitask variational strategy?
# TODO make class that satisfies above requirements

model = MultitaskGPModel(X=x,
                         y=y,
                         likelihood=likelihood,
                         mean_module=mean_module,
                         covar_module=covar_module,
                         num_latents=y.size(-1),
                         task_indices=task_indices)
pred = model(x, task_indices=task_indices)
# model.fit(n_iter=100, lr=0.1, verbose=True)

In [20]:
num_latents, task_indices.max() + 1

(640, tensor(8))

In [29]:
x.shape, y.shape

(torch.Size([640, 5]), torch.Size([80, 8]))